# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
import os, json, re
from dotenv import load_dotenv
from typing import List, Dict, Annotated
from pydantic import BaseModel,Field
from datetime import datetime
import pandas as pd

from tavily import TavilyClient

from rich.console import Console
from rich.markdown import Markdown
console=Console()

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStore

import chromadb
from chromadb.api.types import QueryResult

# placeholders for color priting
ENDC = '\33[0m'
RED = '\33[31m'
GREEN = '\33[32m'
YELLOW = '\33[33m'
VIOLET = '\33[35m'
CYAN = '\33[36m'

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [6]:
class retrievedDocument(BaseModel):
    """Pydantic data model for retrieved documents from the vectorstore"""
    Platform: Annotated[str, Field(description="Game platform (PC, Xbox, Wii...)")]
    Name: Annotated[str, Field(description="Title of the game")]
    YearOfRelease: Annotated[int, Field(description="Year of release")]
    Description: Annotated[str, Field(description="One-liner game description")]


@tool
def retrieve_game(query:str, num_matches:int=5)->List[retrievedDocument]:
    """
    Semantic search: Finds N results in the vector DB closest to the query
    args:
    - query: a question about game industry. 
    - num_matches (int): Number of matches to return (default is 5) - Use this parameter to increase the number of matches returned      

    You'll receive results as list up to num_matches length. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    num_records = collection.count()
    vectorstore_db = VectorStore(chroma_collection=collection)
    outputs:QueryResult = vectorstore_db.query(
        query_texts=[query],
        n_results=num_matches,          # collect the maximum results   - TODO CHECK LOGIC BECAUSE LARGE CONTEXT
        where_document=None,
        where=None
        )

    if outputs:
        metadatas = outputs['metadatas'][0]
        platforms=[];names=[];years=[];descriptions=[]
        # Use pydantic data model
        # Now you can access them as model objects, ie result.Name, result.Description, ect...
        documents = [retrievedDocument(**metadata) for metadata in metadatas]
        
    return documents


@tool
def retrieve_all_games()->List[retrievedDocument]:
    """
    Retrieves all records from the database
    Output:
    You'll receive all records as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    chroma_client = chromadb.PersistentClient(path="chromadb")
    collection = chroma_client.get_collection("udaplay")
    # Retrieve all records
    all_records = collection.get()

    if all_records:
        metadatas = all_records['metadatas']
        platforms=[];names=[];years=[];descriptions=[]
        # Use pydantic data model
        # Now you can access them as model objects, ie result.Name, result.Description, ect...
        documents = [retrievedDocument(**metadata) for metadata in metadatas]
        
    return documents  


# Database overview

In [33]:
# Access each list item as model objects, ie result.Name, result.Description, ect...
all_records = retrieve_all_games()
console.print(f"{len(all_records)} retrieved from the database. Examples: ", style='bold yellow')
all_records[:3]

48 retrieved from the database. Examples: 

[retrievedDocument(Platform='PlayStation 1', Name='Gran Turismo', YearOfRelease=1997, Description='A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.'),
 retrievedDocument(Platform='PlayStation 2', Name='Grand Theft Auto: San Andreas', YearOfRelease=2004, Description="An expansive open-world game set in the fictional state of San Andreas, following the story of Carl 'CJ' Johnson."),
 retrievedDocument(Platform='PlayStation 3', Name='Gran Turismo 5', YearOfRelease=2010, Description='A comprehensive racing simulator featuring a vast selection of vehicles and tracks, with realistic driving physics.')]

In [34]:
temp={}
for i in range(len(all_records)):
    temp[i] = pd.DataFrame(json.loads(all_records[i].model_dump_json()), index=[i])
records_df = pd.concat(temp,ignore_index=True)
records_df

,Platform,Name,YearOfRelease,Description
0,PlayStation 1,Gran Turismo,1997,A realistic racing simulator featuring a wide ...
1,PlayStation 2,Grand Theft Auto: San Andreas,2004,An expansive open-world game set in the fictio...
2,PlayStation 3,Gran Turismo 5,2010,A comprehensive racing simulator featuring a v...
3,PlayStation 4,Marvel's Spider-Man,2018,An open-world superhero game that lets players...
4,PlayStation 5,Marvel's Spider-Man 2,2023,"The sequel to the acclaimed Spider-Man game, f..."
5,Game Boy Color,Pokémon Gold and Silver,1999,Second-generation Pokémon games introducing ne...
6,Game Boy Advance,Pokémon Ruby and Sapphire,2002,Third-generation Pokémon games set in the Hoen...
7,Super Nintendo Entertainment System (SNES),Super Mario World,1990,A classic platformer where Mario embarks on a ...
8,Nintendo 64,Super Mario 64,1996,A groundbreaking 3D platformer that set new st...
9,GameCube,Super Smash Bros. Melee,2001,A crossover fighting game featuring characters...


In [35]:
table = pd.pivot_table(
    records_df, 
    index=['Platform'], 
    columns=['YearOfRelease'], 
    values=['Name'],
    aggfunc='count',
    fill_value=0,
    margins=True, 
    margins_name="Total" 
    )

table.replace(0,'')

Name                                \
YearOfRelease                              1990 1996 1997 1999 2001 2002 2004   
Platform                                                                        
Game Boy Advance                                                       1        
Game Boy Color                                               1                  
GameCube                                                          1             
Nintendo 64                                        1                            
Nintendo Switch                                                                 
PC                                                                              
PlayStation 1                                           1                       
PlayStation 2                                                               1   
PlayStation 3                                                                   
PlayStation 4                                                                   
PlayStation 5                                                                   
Super Nintendo Entertainment System (SNES)    1                                 
Wii                                                                             
Windows                                                                         
Xbox 360                                                                        
Xbox One                                                                        
Xbox Series X/S                                                                 
Xbox Series X|S                                                                 
Total                                         1    1    1    1    1    1    1   

                                                                               \
YearOfRelease                              2006 2010 2014 2017 2018 2021 2022   
Platform                                                                        
Game Boy Advance                                                                
Game Boy Color                                                                  
GameCube                                                                        
Nintendo 64                                                                     
Nintendo Switch                                              1                  
PC                                                                              
PlayStation 1                                                                   
PlayStation 2                                                                   
PlayStation 3                                      1                            
PlayStation 4                                                     1         1   
PlayStation 5                                                               1   
Super Nintendo Entertainment System (SNES)                                      
Wii                                           1                                 
Windows                                                                         
Xbox 360                                           1                            
Xbox One                                                1                       
Xbox Series X/S                                                        1    1   
Xbox Series X|S                                                        1        
Total                                         1    2    1    1    1    2    3   

                                                                 
YearOfRelease                              2023 2024 2025 Total  
Platform                                                         
Game Boy Advance                                              1  
Game Boy Color                                                1  
GameCube                                                      1  
Nintendo 64                                                   1  
Nintendo Switch

#### Evaluate Retrieval Tool

In [7]:
class evaluationResult(BaseModel):
    """pydantic data model for evaluation result"""
    useful: Annotated[bool, Field(description="whether the documents are useful to answer the question")]
    description: Annotated[str, Field(description="description about the evaluation result")]

@tool
def evaluate_retrieval(question:str, retrieved_docs:List[retrievedDocument])->evaluationResult:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: Full context of documents available to answer the question
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    llm = LLM(
        api_key=OPENAI_API_KEY, 
        model='gpt-4o', 
        temperature=0
        )
    
    system_prompt = ("You are an expert at evaluating other LLM's responses.\n"
    "Your task is to evaluate whether the documents retrieved are useful to answer the question from the user. "
    "Ask yourself whether you can answer the user question based on the information provided."
    )
    
    user_prompt= ("Evaluate this output: "
    f"user question: {question}\n"
    f"retrieved documents:\n{retrieved_docs}"
    )

    messages=[
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    response = llm.invoke(
        input=messages, 
        response_format=evaluationResult
        )

    return response.content   

In [27]:
evaluation = evaluate_retrieval('games from 2023', all_records)

console.print(json.loads(evaluation), style='bold yellow')

{
    'useful': True,
    'description': "The retrieved documents include several games released in 2023, such as 'Marvel's Spider-Man 
2', 'Starfield', 'Street Fighter 6', 'Resident Evil 4 (Remake)', 'The Legend of Zelda: Tears of the Kingdom', 
'Baldur’s Gate 3', 'Diablo IV', 'Assassin’s Creed Mirage', 'Alan Wake II', 'Mortal Kombat 1', 'Lies of P', 'Final 
Fantasy XVI', 'Metroid Prime Remastered', 'Hi-Fi RUSH', and 'Dead Space (Remake)'. These documents are useful for 
answering the user's question about games from 2023, as they provide a list of games released in that year along 
with their descriptions and platforms."
}

#### Game Web Search Tool

In [ ]:
@tool
def web_search(query: str, search_depth: str = "advanced") -> Dict:
    """
    Search the web using Tavily API
    args:
        query (str): Search query
        search_depth (str): Type of search - 'basic' or 'advanced' (default: advanced)
    """
    api_key = os.getenv("TAVILY_API_KEY")
    client = TavilyClient(api_key=api_key)
    
    # Perform the search
    search_result = client.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

### Agent

In [ ]:
tools = [retrieve_game,retrieve_all_games,evaluate_retrieval, web_search]

instructions = (
    "You are an helpful assistant with deep knowledge on computer games. "
    "You answer user request and provide faithful, detailed and structured information. "
    "You first use semantic search in your VectorDB to answer questions on computer games. "
    "Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. "
    "In case the information is not present, you can use the web search tool. "
    "You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. "
    "If you do not know the answer just answer 'I dont know'. "
    "Be concise. Answer with bullet points citing your sources. "
    f"Note: your database contains {chromadb.PersistentClient(path="chromadb").get_collection("udaplay").count()} records on the most successful games"
)

agent = Agent(
    model_name="gpt-4o",
    tools=tools,
    instructions=instructions
)

### Helper functions

In [10]:
def print_messages(messages: List):
    """
    pretty print a list of AI messages with color-code using rich and ANSI
    inputs:
        - list of AI messages
    """
    for m in messages:

        content = m.content

        # If this is a tool response containing retrievedDocument objects, pretty-print them - No color
        if isinstance(content, str) and "retrievedDocument(" in content:
            docs = re.findall(r"retrievedDocument\([^)]+\)", content)
            if docs:
                print(f" -> role = {m.role}, tool_calls = {getattr(m, 'tool_calls', None)}")
                print("    content =")
                for doc in docs:
                    print(f"       {doc}")
            else:
                print(f" -> role = {m.role}, content = {content}, tool_calls = {getattr(m, 'tool_calls', None)}")

        # if this is another tool response, pretty print tool call result - No color
        elif isinstance(content, str) and m.role=="tool" :
            print(f" -> role = {m.role}, content = {json.loads(content)}, tool_calls = {getattr(m, 'tool_calls', None)}")

        # if this is a tool calling, print in CYAN color
        elif m.role=="assistant" and m.tool_calls:
            print(CYAN, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}", ENDC)

        # if this is a system prompt, print in VIOLET color
        elif m.role=="system" :
            print(VIOLET, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}", ENDC)

        # if this is the initial user prompt, print in GREEN color
        elif m.role=="user":
            print(GREEN, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}", ENDC)

        # print in YELLOW all generated response by the assistant
        else:
            print(YELLOW, f" -> role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)}",ENDC)

        print('-----------------------------------------------------------')

    # Final message is the assistant's last content - Print final resposne in GREEN color
    if messages:
        last_msg = messages[-1]
        console.print('Final Answer:', style='bold green')
        console.print(Markdown(last_msg.content), style='bold green')

In [11]:
def process_question(question:str, session_id:str=None)->None:
    """
    LLM agent answers question in a specified session
    Workflow messages are printed in addition to final response
    inputs:
        - question from the user (str)
        - session_id (Optional[str])

    outputs:
        - full list of messages from the session_id
    """
    response = agent.invoke(
        query=question,
        session_id=session_id
        )

    print(f"\nMessages from {session_id or 'default'}:")
    messages = response.get_final_state()["messages"]
    print_messages(messages)

    return messages

## Querying agent

- question 1

In [22]:
query = "When Pokémon Gold and Silver was released?"
response = agent.invoke(
    query=query,
    session_id='question_0'
    )

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [49]:
print("\nMessages from run 1:")
messages = response.get_final_state()["messages"]
print_messages(messages)


Messages from run 1:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in you VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet points citing your sources., tool_calls = None 
-----------------------------------------------------------
  -> role = user, content = When Pokémon Gold and Silver was released?, tool_calls = None 
-----------------------------------------------------------
  -> role = assistant, content = None, tool_calls = [ChatCompletionMessageToolC

Final Answer:

Pokémon Gold and Silver was released in 1999 for the Game Boy Color. These games are known for introducing new     
regions, Pokémon, and gameplay mechanics.                                                                          

- question 2

In [50]:
query = "Which one was the first 3D platformer Mario game?"
response = agent.invoke(
    query=query,
    session_id='question_2'
    )

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [51]:
print("\nMessages from run 2:")
messages = response.get_final_state()["messages"]
print_messages(messages)


Messages from run 2:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in you VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet points citing your sources., tool_calls = None 
-----------------------------------------------------------
  -> role = user, content = Which one was the first 3D platformer Mario game?, tool_calls = None 
-----------------------------------------------------------
  -> role = assistant, content = None, tool_calls = [ChatCompletionMessa

Final Answer:

The first 3D platformer Mario game is:                                                                             

 • Game Title: Super Mario 64                                                                                      
 • Platform: Nintendo 64                                                                                           
 • Year of Release: 1996                                                                                           
 • Description: A groundbreaking 3D platformer that set new standards for the genre, featuring Mario's quest to    
   rescue Princess Peach.                                                                                          

This game is widely regarded as a pivotal moment in gaming history, establishing the framework for 3D platformers  
that followed.                                                                                                     

- question 3

In [52]:
query = "Was Mortal Kombat X realeased for Playstation 5?"
response = agent.invoke(
    query=query,
    session_id='question_3'
    )

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__


In [53]:
print("\nMessages from run 3:")
messages = response.get_final_state()["messages"]
print_messages(messages)


Messages from run 3:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in you VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet points citing your sources., tool_calls = None 
-----------------------------------------------------------
  -> role = user, content = Was Mortal Kombat X realeased for Playstation 5?, tool_calls = None 
-----------------------------------------------------------
  -> role = assistant, content = None, tool_calls = [ChatCompletionMessag

Final Answer:

Mortal Kombat X was not released specifically for the PlayStation 5 as a native title but is playable on the       
console via backward compatibility. Here are the key points:                                                       

 • Original Release: Mortal Kombat X was originally released on April 14, 2015, for PlayStation 4.                 
 • Backward Compatibility: The game is compatible with PlayStation 5, meaning you can play the PlayStation 4       
   version on the PS5.                                                                                             
 • Performance: While playing on PS5, the game may benefit from improved loading times, but some features available
   on the PS4 might be absent.                                                                                     
 • Upgraded Version: An enhanced version called Mortal Kombat XL was released on March 1, 2016, for PlayStation 4, 
   which included additional content.                                                                              

For more information, you can refer to the ]8;id=644005;https://en.wikipedia.org/wiki/Mortal_Kombat_X\Wikipedia page on Mortal Kombat X]8;;\ or check the ]8;id=356918;https://store.playstation.com/en-us/product/UP1018-CUSA00967_00-MORTALKOMBATX000\PlayStation Store]8;;\.       

- question 4

In [31]:
#agent.reset_session('question_4')
_ = process_question('How many games playable on PC platform in your database? Provide the list.',session_id='question_4')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from question_4:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answer

Final Answer:

Here are the games playable on the PC platform from my database:                                                   

 1 Monster Hunter Wilds                                                                                            
    • Year of Release: 2025                                                                                        
    • Description: A new open-world Monster Hunter with dynamic environments and bird-like mounts for traversal.   
 2 Split Fiction                                                                                                   
    • Year of Release: 2025                                                                                        
    • Description: A co-op game where two players navigate alternating fantasy and sci-fi worlds in a deeply       
      collaborative story.                                                                                         
 3 Battlefield 6                                                                                                   
    • Year of Release: 2025                                                                                        
    • Description: The next mainline entry in the Battlefield series, with modern conflict between NATO and a      
      private military company.                                                                                    
 4 Assassin’s Creed Mirage                                                                                         
    • Year of Release: 2023                                                                                        
    • Description: A return to the stealth-focused roots of Assassin’s Creed, set in 9th-century Baghdad.          
 5 The First Berserker: Khazan                                                                                     
    • Year of Release: 2025                                                                                        
    • Description: A stylized soulslike following a legendary general—celebrated for its art direction and         
      challenging combat.                                                                                          

There are a total of five games listed as playable on the PC platform in my database.                              

- question 5

In [33]:
#agent.reset_session('question_5')
_ = process_question('List all PC and Playstation games you know of. Indicate their year of release',session_id='question_5')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from question_5:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If

Final Answer:

Here is a list of PC and PlayStation games along with their year of release:                                       

                                                PlayStation Games:                                                 

 • Gran Turismo                                                                                                    
    • Platform: PlayStation 1                                                                                      
    • Year of Release: 1997                                                                                        
 • Grand Theft Auto: San Andreas                                                                                   
    • Platform: PlayStation 2                                                                                      
    • Year of Release: 2004                                                                                        
 • Gran Turismo 5                                                                                                  
    • Platform: PlayStation 3                                                                                      
    • Year of Release: 2010                                                                                        
 • Marvel's Spider-Man                                                                                             
    • Platform: PlayStation 4                                                                                      
    • Year of Release: 2018                                                                                        
 • Marvel's Spider-Man 2                                                                                           
    • Platform: PlayStation 5                                                                                      
    • Year of Release: 2023                                                                                        

                                                     PC Games:                                                     

 • Assassin’s Creed Mirage                                                                                         
    • Platform: PC                                                                                                 
    • Year of Release: 2023                                                                                        
 • Battlefield 6                                                                                                   
    • Platform: PC                                                                                                 
    • Year of Release: 2025                                                                                        
 • Monster Hunter Wilds                                                                                            
    • Platform: PC                                                                                                 
    • Year of Release: 2025                                                                                        
 • Split Fiction                                                                                                   
    • Platform: PC                                                                                                 
    • Year of Release: 2025                                                                                        
 • The First Berserker: Khazan                                                                                     
    • Platform: PC                                                                                                 
    • Year of Release: 2025                                                                                        

These entries provide a mix of classic and upcoming titles across these platforms.                                 

- question 6

In [80]:
#agent.reset_session('default')
_ = process_question('how many games published in 2023?')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from default:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering

Final Answer:

 • In 2023, approximately 14,500 video games were published globally.                                              
 • Over 14,000 of these games were released on the Steam platform alone, marking a significant increase in game    
   releases compared to previous years.                                                                            

Sources:                                                                                                           

 • ]8;id=946118;https://en.wikipedia.org/wiki/2023_in_video_games\Wikipedia]8;;\                                                                                                       
 • ]8;id=332102;https://gamerant.com/games-released-steam-2023-record-number/\Gamerant]8;;\                                                                                                        

- question 6 (variant)

In [81]:
#agent.reset_session('default')
_ = process_question('how many games published in 2023 in the database?')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from default:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If yo

Final Answer:

There are 16 games published in 2023 in the database. Here are some of them:                                       

 • Marvel's Spider-Man 2 (PlayStation 5)                                                                           
 • Starfield (Xbox Series X/S)                                                                                     
 • Street Fighter 6 (Xbox Series X/S)                                                                              
 • Resident Evil 4 (Remake) (Xbox Series X/S)                                                                      
 • The Legend of Zelda: Tears of the Kingdom (Nintendo Switch)                                                     
 • Baldur’s Gate 3 (PlayStation 5)                                                                                 
 • Diablo IV (PlayStation 4)                                                                                       
 • Assassin’s Creed Mirage (PC)                                                                                    
 • Alan Wake II (PlayStation 5)                                                                                    
 • Mortal Kombat 1 (PlayStation 5)                                                                                 

These are just a few examples from the list.                                                                       

In [11]:
#agent.reset_session('question_7')
_ = process_question('Is there a PC game published in 2024?',session_id='question_7')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from question_7:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If

Final Answer:

Yes, there is a PC game published in 2024:                                                                         

 • Helldivers 2                                                                                                    
    • Platform: Windows                                                                                            
    • Year of Release: 2024                                                                                        
    • Description: A co-op sci-fi shooter sequel where players fight to “spread managed democracy” across hostile  
      planets.                                                                                                     

This information confirms the existence of a PC game released in 2024.                                             

# Multiturn Conversation
- agent handles multiple queries in a session, remembering previous context

In [48]:
agent.reset_session('conversation')
_ = process_question(question='Which game was published in 2017 on Nintendo Switch', session_id='conversation')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from conversation:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. 

Final Answer:

 • Game Name: Mario Kart 8 Deluxe                                                                                  
 • Platform: Nintendo Switch                                                                                       
 • Year of Release: 2017                                                                                           
 • Description: An enhanced version of Mario Kart 8, featuring new characters, tracks, and improved gameplay       
   mechanics.                                                                                                      

This game was published on the Nintendo Switch in 2017.                                                            

- evidence of prior context throug the use of session_id

In [49]:
_ = process_question(question='What is the name of this game?', session_id='conversation')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from conversation:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet points citing your sources. Note: your database contains 48 records on the most successful g

Final Answer:

The name of the game is Mario Kart 8 Deluxe.                                                                       

- evidence of info fetch on the internet as the metadata publisher is not retreived from the db

In [50]:
messages = process_question(question='Can you tell me its publisher?', session_id='conversation')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from conversation:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. 

Final Answer:

 • Game Name: Mario Kart 8 Deluxe                                                                                  
 • Publisher: Nintendo                                                                                             

Mario Kart 8 Deluxe was published by Nintendo for the Nintendo Switch in 2017.                                     

In [51]:
_ = process_question(question='List all games from 2023 found in the db? only provide the names', session_id='conversation')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from conversation:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet po

Final Answer:

Here are the games from 2023 found in the database:                                                                

 • Marvel's Spider-Man 2                                                                                           
 • Starfield                                                                                                       
 • Street Fighter 6                                                                                                
 • Resident Evil 4 (Remake)                                                                                        
 • The Legend of Zelda: Tears of the Kingdom                                                                       
 • Baldur's Gate 3                                                                                                 
 • Diablo IV                                                                                                       
 • Assassin's Creed Mirage                                                                                         
 • Alan Wake II                                                                                                    
 • Mortal Kombat 1                                                                                                 
 • Lies of P                                                                                                       
 • Final Fantasy XVI                                                                                               
 • Metroid Prime Remastered                                                                                        
 • Hi-Fi RUSH                                                                                                      
 • Dead Space (Remake)                                                                                             

In [52]:
messages = process_question(question='Which of those are on PlayStation?', session_id='conversation')

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from conversation:
  -> role = system, content = You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet points citing your sources. Note: your database contains 48 records on the most successful g

Final Answer:

Here are the games from 2023 that are available on PlayStation:                                                    

 • Marvel's Spider-Man 2                                                                                           
 • Baldur's Gate 3                                                                                                 
 • Diablo IV                                                                                                       
 • Alan Wake II                                                                                                    
 • Mortal Kombat 1                                                                                                 
 • Lies of P                                                                                                       
 • Final Fantasy XVI                                                                                               
 • Dead Space (Remake)                                                                                             

- Check
    - note the duplicated Marvel's Spider-Man 2 apprearing twice

In [55]:
records_df[records_df.YearOfRelease==2023].sort_values('Platform').reset_index(drop=True).drop_duplicates()

,Platform,Name,YearOfRelease,Description
0,Nintendo Switch,The Legend of Zelda: Tears of the Kingdom,2023,"A sequel to Breath of the Wild, expanding Hyru..."
1,Nintendo Switch,Metroid Prime Remastered,2023,"A remastered edition of the GameCube classic, ..."
2,PC,Assassin’s Creed Mirage,2023,A return to the stealth-focused roots of Assas...
3,PlayStation 4,Diablo IV,2023,The next entry in Blizzard’s dark action RPG s...
4,PlayStation 5,Marvel's Spider-Man 2,2023,"The sequel to the acclaimed Spider-Man game, f..."
5,PlayStation 5,Baldur’s Gate 3,2023,A Dungeons & Dragons-based RPG offering branch...
6,PlayStation 5,Alan Wake II,2023,A psychological horror sequel continuing Alan ...
7,PlayStation 5,Mortal Kombat 1,2023,"A reboot of the Mortal Kombat timeline, featur..."
8,PlayStation 5,Lies of P,2023,"A Soulslike reimagining of the Pinocchio tale,..."
9,PlayStation 5,Marvel’s Spider-Man 2,2023,A continuation of Insomniac’s Spider-Man serie...


In [64]:
records_df[(records_df.YearOfRelease==2023) & (records_df.Platform.str.contains('PlayStation'))].sort_values('Platform')

,Platform,Name,YearOfRelease,Description
24,PlayStation 4,Diablo IV,2023,The next entry in Blizzard’s dark action RPG s...
4,PlayStation 5,Marvel's Spider-Man 2,2023,"The sequel to the acclaimed Spider-Man game, f..."
23,PlayStation 5,Baldur’s Gate 3,2023,A Dungeons & Dragons-based RPG offering branch...
26,PlayStation 5,Alan Wake II,2023,A psychological horror sequel continuing Alan ...
27,PlayStation 5,Mortal Kombat 1,2023,"A reboot of the Mortal Kombat timeline, featur..."
28,PlayStation 5,Lies of P,2023,"A Soulslike reimagining of the Pinocchio tale,..."
29,PlayStation 5,Marvel’s Spider-Man 2,2023,A continuation of Insomniac’s Spider-Man serie...
30,PlayStation 5,Final Fantasy XVI,2023,"A darker, more mature Final Fantasy entry with..."
33,PlayStation 5,Dead Space (Remake),2023,A complete remake of the 2008 sci-fi horror cl...


In [53]:
messages

[SystemMessage(role='system', content="You are an helpful assistant with deep knowledge on computer games. You answer user request and provide faithful, detailed and structured information. You first use semantic search in your VectorDB to answer questions on computer games. Always reason about the response, rephrase the question and call the tool again until you get a satisfiying result. In case the information is not present, you can use the web search tool. You always check the information with the evaluate_retrieval tool before answering the question. Make sure to run the evaluation tool just before your final response. If you do not know the answer just answer 'I dont know'. Be concise. Answer with bullet points citing your sources. Note: your database contains 48 records on the most successful games"),
 UserMessage(role='user', content='Which game was published in 2017 on Nintendo Switch'),
 AIMessage(role='assistant', content=None, tool_calls=[ChatCompletionMessageToolCall(id='c